In [1]:
from selenium import webdriver
import time
import pandas as pd

In [2]:
# input URL for IKEA article
URL = 'https://www.ikea.com/us/en/p/blahaj-soft-toy-shark-90373590/'

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get(URL)

In [5]:
time.sleep(5)

In [6]:
# accept cookies (ensures smooth operation)
driver.find_elements_by_xpath('/html/body/div[8]/div/div[2]/button')[0].click()

In [7]:
# ascertain number of reviews
n_reviews = driver.find_elements_by_class_name('range-revamp-average-rating__reviews')[0].text
n_reviews = int(n_reviews[1:len(n_reviews)-1])
print(n_reviews)

159


In [8]:
# ascertain how many pages of reviews are to be looped over
pages = n_reviews//20
remainder = n_reviews%20

In [9]:
# open Reviews
driver.find_elements_by_xpath('//*[@id="content"]/div/div[1]/div/div[2]/div[2]/div[2]/div[3]/button/span')[0].click()

In [10]:
ratings = []
names_dates = []
titles = []
reviews = []

star = 'range-revamp-svg-icon range-revamp-ratings-bar__star range-revamp-ratings-bar__star--filled'
right_arrow = '//*[@id="range-modal-mount-node"]/div/div[3]/div/div[2]/div/div[2]/div[22]/button[2]'

In [11]:
time.sleep(5)

In [12]:
for i in range(pages):
    for x in range(2,22):
        xpath_prefix = '//*[@id="range-modal-mount-node"]/div/div[3]/div/div[2]/div/div[2]'
        xpath_base = xpath_prefix + '/div[' + str(x) + ']'
    
        # the rating
        xpath_rating_base = xpath_base + '/div[1]/div[1]/span/'
        rating = 0
        for i in range(1,6):
            xpath_rating = xpath_rating_base + '*[name()="svg"][' + str(i) + ']'
            test_element = driver.find_elements_by_xpath(xpath_rating)[0]
            if test_element.get_attribute('class') == star:
                rating += 1
        ratings.append(rating)
    
        # the name and date
        xpath_name_date = xpath_base + '/div[1]/div[2]'
        element_name_date = driver.find_elements_by_xpath(xpath_name_date)[0]
        name_date = element_name_date.text
        names_dates.append(name_date)
    
        # the title
        xpath_title = xpath_base + '/div[2]'
        try:
            element_title = driver.find_elements_by_xpath(xpath_title)[0]
            title = element_title.text
            titles.append(title)
        except:
            titles.append('n/a')
    
        # the review
        xpath_review = xpath_base + '/div[3]'
        try:
            element_review = driver.find_elements_by_xpath(xpath_review)[0]
            review = element_review.text
            reviews.append(review)
        except:
            reviews.append('n/a')
    
    print(len(ratings))
    # next 20
    driver.find_elements_by_xpath(right_arrow)[0].click()
    time.sleep(2.5)

20
40
60
80
100
120
140


In [13]:
# last however many
for x in range(2,remainder+2):
    xpath_prefix = '//*[@id="range-modal-mount-node"]/div/div[3]/div/div[2]/div/div[2]'
    xpath_base = xpath_prefix + '/div[' + str(x) + ']'
    
    # the rating
    xpath_rating_base = xpath_base + '/div[1]/div[1]/span/'
    rating = 0
    for i in range(1,6):
        xpath_rating = xpath_rating_base + '*[name()="svg"][' + str(i) + ']'
        test_element = driver.find_elements_by_xpath(xpath_rating)[0]
        if test_element.get_attribute('class') == star:
            rating += 1
    ratings.append(rating)
    
    # the name and date
    xpath_name_date = xpath_base + '/div[1]/div[2]'
    element_name_date = driver.find_elements_by_xpath(xpath_name_date)[0]
    name_date = element_name_date.text
    names_dates.append(name_date)
    
    # the title
    xpath_title = xpath_base + '/div[2]'
    try:
        element_title = driver.find_elements_by_xpath(xpath_title)[0]
        title = element_title.text
        titles.append(title)
    except:
        titles.append('n/a')
    
    # the review
    xpath_review = xpath_base + '/div[3]'
    try:
        element_review = driver.find_elements_by_xpath(xpath_review)[0]
        review = element_review.text
        reviews.append(review)
    except:
        reviews.append('n/a')
    
print(len(ratings))

159


In [14]:
# Clean up author and date data
authors = []
dates = []

for x in names_dates:
    index = x.find(' - ')
    authors.append(x[0:index])
    dates.append(x[index+3:])

In [15]:
# put data in Pandas data frame
data_dict = {'Author':authors, 'Date': dates, 'Rating':ratings, 'Title':titles, 'Review':reviews}
data_df = pd.DataFrame(data_dict)
data_df

,Author,Date,Rating,Title,Review
0,JINNIE,01/21/2021,5,This is my fourth purchase,This is my fourth purchase and I need them all...
1,Robbie,01/14/2021,5,Shark is number one,Everyone loves the shark. I hope you carry it ...
2,MARGORY,01/13/2021,5,My daughter loves,My daughter loves
3,DENISE,01/11/2021,5,Grandson loves it!,Grandson loves it!
4,Angelica,01/11/2021,5,My nephew loved it!,My nephew loved it!
...,...,...,...,...,...
154,nana24,10/16/2017,5,another fantastic IKEA stuffed toy!,"Purchased 4 of these, one for each grandchild ..."
155,Tillybean,10/16/2017,5,"Love, love, love it!",This stuffed toy is so soft and cuddly! Everyo...
156,ErtBert,09/12/2017,5,Super soft!,I got this on an impulse and I have loved it e...
157,Blahajluv,08/31/2017,5,I finally found him!,I love sharks and I missed out on buying him y...


In [16]:
# write out df to csv
data_df.to_csv('reviews.csv', index=False)